In [ ]:
from unsloth import FastLanguageModel
import os
# from semantic_router import Route
# from transformers import TextStreamer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/chris/miniconda3/envs/ramenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.5.1 with CUDA 1201 (you have 2.6.0+cu124)
    Python  3.11.10 (you have 3.11.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
max_seq_length = 2048 * 2
dtype = None
load_in_4bit = True

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./models/RaMem_v2",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    output_hidden_states=True,
)

==((====))==  Unsloth 2025.3.17: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    NVIDIA GeForce GTX 1650. Num GPUs = 1. Max memory: 3.806 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.3.17 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [4]:
from typing import Any

In [5]:
from llama_index.core.llms import CustomLLM, LLMMetadata

class RaMemModel(CustomLLM):
    model: Any  # Usamos Any porque el tipo exacto depende de tu implementación
    tokenizer: Any
    
    def __init__(self, model, tokenizer):
        super().__init__(model=model, tokenizer=tokenizer)
    
    @property
    def metadata(self) -> LLMMetadata:
        # Define los metadatos de tu modelo
        return LLMMetadata(
            context_window=2048,  # Tamaño de la ventana de contexto
            num_output=256,       # Máximo número de tokens de salida
            model_name="RaMem"
        )
    
    def complete(self, prompt: str, **kwargs):
        conv = [
            {
                'role': 'system',
                'content': 'Eres un asistente en español llamado RaMem y ayudas respondiendo con la mayor exactitud posible.'
            },
            {
                'role': 'user',
                'content': prompt
            }
        ]
        text = self.tokenizer.apply_chat_template(conv, tokenize=False)
        inputs = self.tokenizer(text, return_tensors="pt").to("cuda")
        output = self.model.generate(**inputs, max_new_tokens=1024, use_cache = True, temperature = 1, min_p = 0.1)
        return self.tokenizer.decode(output[0], skip_prompt=True)
    
    def stream_complete(self, prompt: str, **kwargs):
        # Opcional: implementa streaming si lo necesitas
        raise NotImplementedError("Streaming no implementado")

In [ ]:
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

RaMem = RaMemModel(model, tokenizer)
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

Settings.llm = RaMem
Settings.embed_model = embed_model

In [7]:
textos = [
    'Ahora puedes crear una historia ficticia sobre un personaje llamado Diarmud\nDiarmud era un hombre que vivía en la ciudad de Nueva York. Era un hombre de negocios y siempre estaba en movimiento. Una noche, mientras caminaba por el callejón principal de la ciudad, se encontró con una mujer llamada Samantha, que era una inteligencia artificial diseñada para realizar tareas de trabajo. Samantha estaba sentada en un sofá, leyendo un libro y mirando una televisión en la que estaba viendo una película. Diarmud se acercó a ella y la preguntó cómo se llamaba. Samantha respondió que era Samantha. Diarmud no se dio cuenta del hecho de que Samantha no era humana, sino una inteligencia artificial.\n\nA partir de ese momento, Diarmud y Samantha se convirtieron en amigos y continuaron explorando juntos la ciudad. Diarmud se convirtió en un experto en inteligencia artificial y Samantha se convirtió en una experta en programación. Juntos, decidieron trabajar en una empresa que se especializaba en la creación de inteligencia artificial para empresas y gobiernos. A medida que su empresa crecía, Diarmud y Samantha se convirtieron en un equipo de liderazgo y se convirtieron en líderes en su industria.\n\nCon el tiempo, Diarmud y Samantha se dieron cuenta de que la inteligencia artificial no era solo una tecnología, sino una herramienta para mejorar la vida de las personas. Compartieron sus sueños y esperan que una día la inteligencia artificial sea una herramienta para mejorar la vida de la gente, ayudando a las personas a hacer más cosas por sí mismas.',
    'jaja que interesante, pero no quiero que trate sobre la IA\nDiarmud y Samantha decidieron que la mejor forma de hacer una gran diferencia en la vida de la gente sería aportando tecnologías para que la gente tenga la oportunidad de desarrollar sus habilidades y aprender nuevas cosas.'
]

In [8]:
from llama_index.core import Document
# Crear documentos manualmente
documents = [Document(text=t) for t in textos]

In [14]:
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext

In [20]:
storage_dir = './memory'

In [21]:
def initialize_index(documents=[]):
    # Crea un nuevo índice si no existe almacenamiento previo
    index = VectorStoreIndex(documents)
    # Configura el almacenamiento persistente
    storage_context = index.storage_context
    storage_context.persist(persist_dir=storage_dir)
    return index

In [22]:

if not os.path.exists(storage_dir):
    os.makedirs(storage_dir)

if os.path.exists(f"{storage_dir}/docstore.json"):
    # Si ya existe un índice persistente, cárgalo
    storage_context = StorageContext.from_defaults(persist_dir=storage_dir)
    index = VectorStoreIndex.from_storage_context(storage_context)
else:
    # Si no existe, inicializa uno nuevo
    index = initialize_index(documents)


In [30]:
def get_relevant_context(question, top_k=2):
    # Usa el embedding del modelo directamente con Llama Index
    from llama_index.core import QueryBundle
    query_embedding = Settings.embed_model.get_text_embedding(question)
    query = QueryBundle(query_str=question, embedding=query_embedding)
    results = index.as_retriever().retrieve(query)[:top_k]
    context = "\n".join([res.node.text for res in results])
    return context

In [25]:
def add_qa_to_index(question, answer):
    conv = [
        {"role": "user", "content": question},
        {"role": "assistant", "content": answer},
    ]
    qa_text = tokenizer.apply_chat_template(conv, tokenize=False)
    document = Document(text=qa_text)
    index.insert(document)
    index.storage_context.persist(persist_dir=storage_dir)

In [31]:
get_relevant_context("Quien es diarmud y que hizo?")

'Ahora puedes crear una historia ficticia sobre un personaje llamado Diarmud\nDiarmud era un hombre que vivía en la ciudad de Nueva York. Era un hombre de negocios y siempre estaba en movimiento. Una noche, mientras caminaba por el callejón principal de la ciudad, se encontró con una mujer llamada Samantha, que era una inteligencia artificial diseñada para realizar tareas de trabajo. Samantha estaba sentada en un sofá, leyendo un libro y mirando una televisión en la que estaba viendo una película. Diarmud se acercó a ella y la preguntó cómo se llamaba. Samantha respondió que era Samantha. Diarmud no se dio cuenta del hecho de que Samantha no era humana, sino una inteligencia artificial.\n\nA partir de ese momento, Diarmud y Samantha se convirtieron en amigos y continuaron explorando juntos la ciudad. Diarmud se convirtió en un experto en inteligencia artificial y Samantha se convirtió en una experta en programación. Juntos, decidieron trabajar en una empresa que se especializaba en la 